In [1]:
from load_results import load_result_dataset
import pandas as pd

pn1 = 'full_fine_tuning_50epochs_edge_paper_final2'
pn2 = 'full_fine_tuning_50epochs_paper_final2'
pn3 = 'none'
final_data1 = load_result_dataset(pn1, pn2, pn3)
final_data1 = [{**d, 'ft_strategy': 'FFT (50 epochs)'} for d in final_data1]
df1 = pd.DataFrame(final_data1)

pn1 = 'full_fine_tuning_5epochs_edge_article1'
pn2 = 'full_fine_tuning_5epochs_article1'
pn3 = 'none'
final_data2 = load_result_dataset(pn1, pn2, pn3)
final_data2 = [{**d, 'ft_strategy': 'FFT (5 epochs)'} for d in final_data2]
df2 = pd.DataFrame(final_data2)

pn1 = 'linearprobe_50epochs_edge_paper_final2'
pn2 = 'linearprobe_50epochs_paper_final2'
pn3 = 'none'
final_data3 = load_result_dataset(pn1, pn2, pn3)
final_data3 = [{**d, 'ft_strategy': 'LP (50 epochs)'} for d in final_data3]
df3 = pd.DataFrame(final_data3)

df = pd.concat([ df1, df2, df3 ], axis=0, ignore_index=True) 

final_data = []
final_data.extend(final_data1)
final_data.extend(final_data2)
final_data.extend(final_data3)

../results/none/CLIP-convnext_base_w-laion_aesthetic-s13B-b82K_uc-merced-land-use-dataset_TRADES_v2.pkl
HEY
../results/full_fine_tuning_50epochs_paper_final2/CLIP-convnext_base_w-laion_aesthetic-s13B-b82K_uc-merced-land-use-dataset_TRADES_v2.pkl
../results/none/CLIP-convnext_base_w-laion2B-s13B-b82K_uc-merced-land-use-dataset_TRADES_v2.pkl
HEY
../results/full_fine_tuning_50epochs_paper_final2/CLIP-convnext_base_w-laion2B-s13B-b82K_uc-merced-land-use-dataset_TRADES_v2.pkl
../results/none/deit_small_patch16_224.fb_in1k_uc-merced-land-use-dataset_TRADES_v2.pkl
HEY
../results/full_fine_tuning_50epochs_paper_final2/deit_small_patch16_224.fb_in1k_uc-merced-land-use-dataset_TRADES_v2.pkl
../results/none/robust_resnet50_uc-merced-land-use-dataset_TRADES_v2.pkl
HEY
../results/full_fine_tuning_50epochs_paper_final2/robust_resnet50_uc-merced-land-use-dataset_TRADES_v2.pkl
../results/none/vit_small_patch16_224.augreg_in21k_uc-merced-land-use-dataset_TRADES_v2.pkl
HEY
../results/full_fine_tuning_50

In [2]:
df = pd.DataFrame(final_data)
# df.to_csv("./results_dataset.csv")

In [4]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from scipy.stats import spearmanr, linregress
import numpy as np

# Assumes df is already defined
from process_database import process_rankings, global_grouped_dataset

grouped_df = process_rankings(global_grouped_dataset(df))
grouped_df.columns = ['_'.join(col).strip() if isinstance(col, tuple) else col for col in grouped_df.columns]
grouped_df = grouped_df.reset_index()

sub_df = grouped_df[["ft_strategy", "model_type", "model_size", "backbone", "loss_function", "TOTAL_score_sum"]]

pivot_df = sub_df.pivot_table(
    index=["backbone", "loss_function", "model_type", "model_size"],
    columns="ft_strategy",
    values="TOTAL_score_sum"
).reset_index()

symbol_map = {"TRADES": "diamond", "Classic AT": "star"}
color_map = {"TRADES": "#E69F00", "Classic AT": "#56B4E9"}  # Assign colors by loss_function

for (x1, x2) in [("FFT (5 epochs)", "FFT (50 epochs)"), ("LP (50 epochs)", "FFT (50 epochs)")]:
    pivot_df = pivot_df.fillna(0)
    x, y = pivot_df[x1], pivot_df[x2]
    r_global, p_global = spearmanr(x, y)
    m_global, b_global, *_ = linregress(x, y)
    x_range = np.linspace(x.min(), x.max(), 100)

    # Base scatter
    fig = px.scatter(
        pivot_df, x=x1, y=x2,
        color="loss_function",
        symbol="loss_function",
        color_discrete_map=color_map,
        symbol_map=symbol_map,
    )

    # Global regression line
    fig.add_trace(go.Scatter(
        x=x_range,
        y=m_global * x_range + b_global,
        mode="lines",
        line=dict(color="black", width=2),
        name=f"Global (r={r_global:.2f}, p={p_global:.3f})"
    ))

    # Per-loss-function lines
    for loss_fn in pivot_df["loss_function"].unique():
        subset = pivot_df[pivot_df["loss_function"] == loss_fn]
        if len(subset) < 2:
            continue
        xg, yg = subset[x1], subset[x2]
        rg, pg = spearmanr(xg, yg)
        mg, bg, *_ = linregress(xg, yg)
        fig.add_trace(go.Scatter(
            x=x_range,
            y=mg * x_range + bg,
            mode="lines",
            line=dict(color=color_map[loss_fn], dash="dot"),
            name=f"{loss_fn} (r={rg:.2f}, p={pg:.3f})"
        ))

    fig.update_layout(
        margin=dict(l=0, r=0, t=0, b=0),
        width=800, height=400,
        plot_bgcolor="white", paper_bgcolor="white",
        xaxis=dict(
            title=f"{x1} (Sum Score)",
            showgrid=True,
            gridcolor="lightgrey",
            range=[-0.25, 8.25]
        ),
        yaxis=dict(
            title=f"{x2} (Sum Score)",
            showgrid=True,
            gridcolor="lightgrey",
            range=[0, 20]
        ),
        legend=dict(
            font=dict(size=12),
            y=0.85,
            bgcolor="rgba(255,255,255,1)",
            bordercolor="lightgrey",
            borderwidth=1
        )
    )


    fig.update_traces(marker=dict(size=8, line=dict(width=0)))
    fig.show()
    fig.write_image(f"./paper_figures/low_cost_proxy_{x1.replace(' ', '_')}.png", scale=3)
